In [ ]:
import math

def reward_function(params):
    '''
    Combined reward function with parameters for: center line following, 
    avoiding excessive steering (preventing zigzags), keeping wheels on 
    the track, and speed management using waypoints to calculate the curves 
    of the oncoming track.
    '''
    # ------------------------------------------------------------------------
    # Limiting Factors
    max_speed = 3.5  # Maximum speed (used for straight portions of the track)
    min_speed = 1.0  # Minimum speed (used for turns)
    abs_steering_threshold = 15 # Where |15| or |-15| degrees = a indicator 
                                # for a turn (car is no longer going straight)
                                # Max turn radius pre-set to -30 to 30 degrees
    track_curve_threshold = math.radians(10) # Where 10 radians = indicator
                                             # seperating curves & straight track
                                             # radians bc/ math.atan2 uses raidans
                                 
    # ------------------------------------------------------------------------
    # Initialize reward
    reward = 1e-3  # Start with a very low reward by default

    # ------------------------------------------------------------------------
    # Read input parameters
    track_width = params['track_width']
    distance_from_center = params['distance_from_center']
    all_wheels_on_track = params['all_wheels_on_track']
    speed = params['speed']
    abs_steering = abs(params['steering_angle'])
    waypoints = params['waypoints']
    closest_waypoints = params['closest_waypoints']

    # ------------------------------------------------------------------------
    # Define 3 markers for distance from the center
    marker_1 = 0.1 * track_width
    marker_2 = 0.25 * track_width
    marker_3 = 0.5 * track_width

    # Distance from center rewards
    # Give the largest reward for staying close to the centre
    if distance_from_center <= marker_1:
        reward = 1.5
    elif distance_from_center <= marker_2:
        reward = 0.5
    elif distance_from_center <= marker_3:
        reward = 0.1

    # ------------------------------------------------------------------------
    # Steering penalty to avoid zig-zag
    if abs_steering > abs_steering_threshold:
        reward *= 0.8
    
    # ------------------------------------------------------------------------
    # Wheels on track check
    if all_wheels_on_track and (0.5*track_width - distance_from_center) >= 0.05:
        reward = 1.0
        
    # Penalize heavily if the car goes off-track
    if not all_wheels_on_track:
        reward = 1e-3
        
    # ------------------------------------------------------------------------
    # Using waypoints to calculate the curves of the oncoming track
    # to manage the speed
    
    # Part 1. Waypoints
    
    # Coordinates for the current, next, next_next waypoints
    current_waypoint = waypoints[closest_waypoints[0]] # index 0 for closest
    next_waypoint = waypoints[closest_waypoints[1]] # index 1 for next closest
    next_next_waypoint_index = (closest_waypoints[1] + 1) % len(waypoints)
    next_next_waypoint = waypoints[next_next_waypoint_index]
    
    # NOTE: % len(waypoints) is used to avoid index errors.
    # Ex. if next_waypoint is at index 9, without the modulus, 
    # next_next_waypoint = (9+1) = 10, but index range = (0 to 9) = error.
    # With modulus: next_next_waypoint = (9+1) % 10 = 0 (index wrap back to 0)

    # ------------------------------------------------------------------------
    # Part 2. Direction vector
    # Using vectors to represent the direction of the oncoming track
    # To determine vectors, we need to obtain x and y coordinates
    
    # NOTE: (x,y) where x = horizontal position, y = vertical position
    #       index (0,1) where x is represented with 0 and y with 1
    
    direction_vector = [next_waypoint[0] - current_waypoint[0], 
                        next_waypoint[1] - current_waypoint[1]]
    
    # Vector x-coordinate: Subtract the x-coordinate [0] of the current waypoint
    # from the x-coordinate of the next waypoint.
    # Vector y-coordinate: Similarly, subtract the y-coordinate [1] of the 
    # current waypoint from the y-coordinate of the next waypoint.
    
    # Repeating the same steps but with the proceeding waypoints x and y coordinates
    next_direction_vector = [next_next_waypoint[0] - next_waypoint[0], 
                             next_next_waypoint[1] - next_waypoint[1]]
    
    # ------------------------------------------------------------------------
    # Part 3. Curves of oncoming track
    
    # Calculate the angle between the two direction vectors
    track_angle = math.atan2(next_direction_vector[1], next_direction_vector[0]) 
                  - math.atan2(direction_vector[1], direction_vector[0])
    
    # .atan2(y,x) returns arc tangent of y/x (in radians) = angle between 
    # x-axis and the direction vector point given (angles covered in all 4Q)
    # Subtract angle of direction_vector from next_direction_vector indicates
    # if the oncoming track has a curve.
    # Difference is 0 = straight path, difference is larger = curve,  
    # difference is even larger = sharp turn
    
    # Since the direction of the curve (left/right) is not important to 
    # speed managment, keep the angles positive for simplicity for calculations
    track_angle = abs(track_angle)
    
    # ------------------------------------------------------------------------
    # Speed management
    if track_angle < track_curve_threshold:
        # Large reward for max speed on straight tracks
        if speed >= max_speed:
            reward *= 1.0
    else:
        # Large reward for slower speed on turns/curves
        if speed <= min_speed:
            reward *= 1.25

    return float(reward)